In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument('-headless')

browser = webdriver.Firefox(options=firefox_options)

url = 'https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=9'
browser.get(url)

dfs = pd.read_html(str(browser.page_source))

# 18.1

In [4]:
dfs[3]

,0,1,2,3
0,Rank,↑ ↓,Fighter,Points
1,201,NR,Adrick Croes,24
2,202,-1,Callyugibrainn Marinho Borges,24
3,203,1,Jhonata Diniz,24
4,204,-1,Christian Larsen,24
5,205,2,Cody East,24
6,206,NR,Shamil Tsakuev,24
7,207,-2,Marsel Sychev,24
8,208,-2,Hassan Yousefi,24
9,209,NR,Phillip Latu,24


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Set options for headless mode
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument('-headless')

# Initialize Firefox WebDriver with headless option
driver = webdriver.Firefox(options=firefox_options)

# URL pattern
base_url = "https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue={}&Division={}&Page={}"

# Create an empty DataFrame
columns = ["snapshot_periods", "weight_classes", "rank", "name", "country", "points"]
data = []


weight_classes_range = range(1, 13)  # define weightclasses to obtain

for division in weight_classes_range:
    issue = 878 # needs any arbitrary issue to get full list of issues
    driver.get(base_url.format(issue, division, 1)) 
    try:
        # Locate and extract snapshot periods from the dropdown
        snapshot_dropdown = Select(driver.find_element(By.NAME, "Issue"))
        snapshot_periods = [option.get_attribute("value") for option in snapshot_dropdown.options if option.get_attribute("value")]
        max_pages_per_snapshot = 25
        # loop through snapshot periods
        for issue in snapshot_periods:
            page = 1
            while page <= max_pages_per_snapshot:  # Limit the number of pages per snapshot
                url = base_url.format(issue, division, page)
                driver.get(url)
                print("Current URL:", url)
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//table[@class='tblRank']//tbody/tr")))
                    rows = driver.find_elements(By.XPATH, "//tbody/tr[td[@class='tdRank']]")
                    rows_alt = driver.find_elements(By.XPATH, "//tbody/tr[td[@class='tdRankAlt']]")
                    print("Number of rows:", len(rows)+len(rows_alt))
                    for row in rows:
                        rank = row.find_element(By.CLASS_NAME, "tdRank").text
                        name = row.find_element(By.CLASS_NAME, "sherLink").text
                        country = row.find_element(By.TAG_NAME, "img").get_attribute("src").split("/")[-1].split(".")[0]
                        points = row.find_element(By.CLASS_NAME, "tdBar").text
                        data.append([issue, division, rank, name, country, points])
                    for row in rows_alt:
                        rank = row.find_element(By.CLASS_NAME, "tdRankAlt").text
                        name = row.find_element(By.CLASS_NAME, "sherLink").text
                        country = row.find_element(By.TAG_NAME, "img").get_attribute("src").split("/")[-1].split(".")[0]
                        points = row.find_element(By.CLASS_NAME, "tdBar").text
                        data.append([issue, division, rank, name, country, points])
                except NoSuchElementException:
                    print("No table rows found on this page.")
                page += 1
    except NoSuchElementException:
            # Handle the case where the element with name "Issue" is not found
        print("Element with name 'Issue' not found on the page")

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=columns)

# Close the WebDriver
driver.quit()

# Print the DataFrame
print(df)